# A Title

### Global Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import time
print("passing suez canal:", end='')
for i in range(5):
    time.sleep(2)
    print(" -", end='')
print(">")

passing suez canal: - - - - ->


## Train test split

In [3]:
from sklearn.model_selection import KFold

In [4]:
X = pd.read_csv("data/recipe_train.csv", header=0)

# change split scale here
kf = KFold(n_splits=20, shuffle=True)
kf.get_n_splits(X)

for a, b in kf.split(X):

    kf = KFold(n_splits=5, shuffle=True)
    kf.get_n_splits(b)

    for c, d in kf.split(b):
        
        train_small = X.iloc[c]
        test_small = X.iloc[d]

        train_small.to_csv("data/train_small.csv", index=False)
        test_small.to_csv("data/test_small.csv", index=False)
        
        print(f"created training set containing {len(train_small.index)} instances")
        print(f"created test set containing {len(test_small.index)} instances")
        
        break
    break

created training set containing 1600 instances
created test set containing 400 instances


## 1R model

In [5]:
from models.model_1R import n_1R, predict_1R

In [6]:
model = n_1R("data/train_small.csv", rule='steps')
predict_1R("data/test_small.csv", model, name='steps_1R_prediction')
model = n_1R("data/train_small.csv", rule='ingredients')
predict_1R("data/test_small.csv", model, name='ingredients_1R_prediction')

training on data/train_small.csv using rule: 'steps'
predicted data/test_small.csv, saved to column 'steps_1R_prediction'
training on data/train_small.csv using rule: 'ingredients'
predicted data/test_small.csv, saved to column 'ingredients_1R_prediction'


### *#placeholder*

## Evaluation

In [7]:
from utilities.evaluation import simple_accuracy

In [8]:
simple_accuracy("data/test_small.csv")

accuracy for 'steps_1R_prediction': 65.50%
accuracy for 'ingredients_1R_prediction': 62.25%
